* Indexing method for making dictionary keys - DONE
* each position is an object with index but also list showing cRDS - DONE
* CAN PASS CARDS TO MC SO NO NEED TO BACK CONVERT. JUST USED AS A KEY. LIST IS USED FOR SCORING AND MC - DONE
* Create full list of positions - DONE
* Function to convert index keys to simulation inputs - DONE
* MC simulation
* Exact analytical solution
* Algo for using simulation to solve for strategy
* Compare analytical and numerical solution
* For playing games, the int indexes may just work. Just for strategy making they wont as those impossible hands are tehcnically possible. 


In [143]:
#Example index
17000000001001
#Example_hand
[7,0,0,0,0,0,0,0,0,10,1]

[7, 0, 0, 0, 0, 0, 0, 0, 0, 10, 1]

In [144]:
def cardsToIndex(player_hand, dealer_card, draw = None):
    index_scaling = [10**11, 10**10, 10**9, 10**8, 10**7, 10**6, 10**5, 10**4, 10**2, 10**0]
    index = 10000000000000 + dealer_card*10**12
    if draw != None:
        player_hand[10 - draw] += 1
    for card in range(10):
        index += index_scaling[card] * player_hand[card]
    return index

In [145]:
def indexToCards(index):#TODO
    index = list(str(index)[1:])
    dealer_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    dealer_hand[9 - int(index[0])] += 1
    index = index[1:]
    return [int(x) for x in index[0:-4] + [index[-4] + index[-3]] + [index[-2] + index[-1]]], dealer_hand

In [146]:
def score(position):
    score = position.player_hand[-1] * 11
    high_aces = position.player_hand[-1]
    for cardIndex in range(9):
        score += (10 - cardIndex) * position.player_hand[cardIndex]
    #Determine whether to use aces as 1 or 11
    while score > 21 and high_aces > 0:
        if high_aces > 0:
            score -= 10
            high_aces -= 1
    return score 

In [147]:
class Position:
    def __init__(self, index = None, cards = None, dealer_card = None, draw = None, decks = 1):
        if index != None:
            self.index = index
            self.player_hand, self.dealer_hand = indexToCards(self.index)
        if index == None:
            self.player_hand = cards
            self.dealer_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            self.dealer_hand[10 - dealer_card] += 1
            self.index = cardsToIndex(self.player_hand, dealer_card, draw)

        self.score = score(self)
        self.bust = True if self.score > 21 else False
        self.decks = decks

        self.cards_in_play = [x + y for x, y in zip(self.player_hand, self.dealer_hand)]

        self.valid = True if all(x < 4*self.decks + 1  for x in self.cards_in_play[1:]) and self.cards_in_play[0] < 16*self.decks + 1 else False

In [152]:
def getPositions(decks = 1):
    Next = {10000000000000 : Position(index = 10000000000000, dealer_card = 1)}
    Master = {}
    runs = 0
    while len(Next) != 0:
        Current = Next
        Next = {}
        for position in Current.values():
            for drawn in range(1, 11):
                gate = True
                for dealer_card in range(1, 11):
                    temp = position.player_hand.copy()
                    current_position = Position(cards = temp, draw = drawn, dealer_card = dealer_card, decks = decks)
                    if not current_position.bust and current_position.valid:
                        if gate:
                            Next[current_position.index] = current_position
                            gate = False
                        if runs > 0:
                            Master[current_position.index] = current_position
        runs += 1
    return Master

In [155]:
len(getPositions(decks = 2))

27851

#### mc

aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']

np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])
array(['pooh', 'pooh', 'pooh', 'Christopher', 'piglet'], # random
      dtype='<U11')

# interesting
Yo umay be able to start simulating games for stick probs but just do all dealer possibilities, then you don't need to od te win check each time

In [173]:
import numpy as np
np.random.choice(range(10), p = np.array(range(10))/sum(range(10)))

9

In [ ]:
def simulateGames(position, number_games = 1000):
    wins, losses, ties = 0, 0, 0
    dealer_card = rn.randint(1,10, number_games)
    player_score = position.score

    draw_weights = np.array([x - y for x, y in zip([card*position.decks for card in [16, 4, 4, 4, 4, 4, 4, 4, 4, 4]], position.cards_in_play)])



    #Get 1

    while number_games > 0:
        temp_weights = draw_weights.copy()
        
    #get scpre
    #if lt 16 go again
        while dealer_score < 17:
            dealer_hand = [dealer_card]
            dealer_hand.append(np.random.choice(range(1, 10), 5, p=temp_weights/sum(temp_weights)))
            temp_weights[10 - card] -= 1
            dealer_score = score(dealer_hand)



#get winner


#fix score thing
#deal with consequences of that shit
#finish mc 


        number_games -= 1 

In [156]:
import random as rn

6

In [169]:
for i in range(10000000):
    rn.randint(1,10)

issue:
I want to pass dealer hand plus a card and get score
I cant because score takes a position
I shouldchang scor to take hadn
now I need to pass a full hand many times

simulate 10 possible deepar draws
then file them
then run through the randint list. ez
Oly check scores once